### Plan

- set up standard operators
- function to make tensor products with trivial operators
- function to check linear independence
- find way to determine dependence?
- function to build closed lie algebras (with cut off)

In [2]:
import numpy as np

In [16]:
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [18]:
np.kron(I, I)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [ ]:
def tensor(W, dim, start, end=0):
    if not end:
        end = start + 1
    if start==0:
        A = W
    else:
        A = I
    for i in range(len(end-start)):
